In [ ]:
IMG_DIR = 'path/to/data/'
CSV_PATH = 'path/to/data/bbox.csv'

In [ ]:
from utils import get_anchor_dict

img_size = 640
anchor_dict = get_anchor_dict(CSV_PATH, img_size)

In [ ]:
# 教師データのDataLoaderの設定
from torch.utils.data import DataLoader

from dataset import YOLOv3_Dataset

img_size_x = 640
img_size_y = 480
transform = transforms.ToTensor()

dataset = YOLOv3_Dataset(IMG_DIR, CSV_PATH, img_size_x, img_size_y, anchor_dict, transform)

In [ ]:
batch_size = 1
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(len(dataset))
print(len(train_loader))

In [ ]:
# dataloaderのテスト
mini_train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# イテレータを作成
iterator = iter(mini_train_loader)

# 1つのバッチを取り出す
img, target3, target2, target1 = next(iterator)

In [ ]:
# 描画テスト
from utils import get_multiple_bboxes

flag, bbox_list, label_list, score_list, NMS_bbox_list, _, NMS_label_list, NMS_score_list = get_multiple_bboxes([target3[0], target2[0], target1[0]], anchor_dict, num_cls = 3, img_size_x, img_size_y)

In [ ]:
# 非最大値抑制適応
draw_bbox(img[0], NMS_bbox_list, NMS_label_list, [])

In [ ]:
# テスト
from model import YOLOv3

model = YOLOv3()
with torch.no_grad():
    output = model(torch.zeros((1,3,640,480)))
for i in range(3):
    print(output[i].shape)

In [ ]:
from utils import YOLOv3_loss_function

from torch import optim

# モデル
model = YOLOv3()
#model.load_state_dict(torch.load('./YOLOv3_weights_e100.pth'))
model.cuda()

# 最適化アルゴリズム
optimizer = optim.Adam(model.parameters())

# エポック数
epoch = 100

In [ ]:
# 損失のログ
record_loss_train = []
record_loss_test = []

for i in range(epoch):
    model.train()
    loss_train = 0
    coord_loss_train = 0
    obj_loss_train = 0
    class_loss_train = 0
    noobj_loss_train = 0
    for j, (img, target3, target2, target1) in enumerate(train_loader):
        img, target3, target2, target1 = img.cuda(), target3.cuda(), target2.cuda(), target1.cuda()  # GPU対応
        pred3, pred2, pred1 = model(img)
        #print(pred3.size())
        #print(target3.size())
        loss, coord_loss, obj_loss, class_loss, noobj_loss= YOLOv3_loss_function([pred3, pred2, pred1], [target3, target2, target1])
        loss_train += loss.item()
        coord_loss_train += coord_loss.item()
        obj_loss_train += obj_loss.item()
        class_loss_train += class_loss.item()
        noobj_loss_train += noobj_loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    coord_loss_train /= j+1
    obj_loss_train /= j+1
    class_loss_train /= j+1
    noobj_loss_train /= j+1
    record_loss_train.append(loss_train)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:",round(loss_train, 3), "coord_loss_train", round(coord_loss_train, 3), "obj_loss_train", round(obj_loss_train, 3), "class_loss_train", round(class_loss_train, 3), "noobj_loss_train", round(noobj_loss_train, 3))


In [ ]:
# 誤差の推移
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train)
#plt.plot(range(len(record_loss_test)), record_loss_test)
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

In [ ]:
model.eval()

transform = transforms.ToTensor()

image = Image.open('path/to/data/xxx.jpg')
image = transform(image)
#print(image.unsqueeze(dim=0).size())
pred3, pred2, pred1 = model(image.unsqueeze(dim=0).cuda())
#print(pred3)
#bbox_list, NMS_bbox_list, _, f= get_multiple_bboxes([pred3[0], pred2[0], pred1[0]], anchor_dict, 1024, 0)
flag, bbox_list, label_list, score_list, NMS_bbox_list, _, NMS_label_list, NMS_score_list = get_multiple_bboxes([pred3[0], pred2[0], pred1[0]], anchor_dict, num_cls = 3)
print(flag)
if flag == True:
    draw_bbox(image, bbox_list)
    draw_bbox(image, NMS_bbox_list)
    print(NMS_label_list)
    print(NMS_score_list)

In [ ]:
torch.save(model.state_dict(), 'path/to/data/xxx.pth')